In [2]:
import torch
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
import random
import os
import sys
import time
from tqdm import tqdm
from google.colab import drive

class BackTranslator:
    def __init__(self, source_lang="en", target_langs=["fr", "de", "es"]):
        """
        Initialize the back-translator with source language and a list of target languages.

        Args:
            source_lang: Source language code (default: "en" for English)
            target_langs: List of target language codes for translation
        """
        self.source_lang = source_lang
        self.target_langs = target_langs
        self.models = {}
        self.tokenizers = {}

        # Set device for inference
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
            print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
        else:
            self.device = torch.device("cpu")
            print("Using CPU (no GPU acceleration available)")

        # Load models for each language pair
        self._load_models()

    def _load_models(self):
        """Load translation models for each language pair."""
        print("Loading translation models...")

        # Load models for translation from source to target languages
        for tgt_lang in self.target_langs:
            model_name = f'Helsinki-NLP/opus-mt-{self.source_lang}-{tgt_lang}'
            print(f"Loading {model_name}...")

            self.models[f"{self.source_lang}-{tgt_lang}"] = MarianMTModel.from_pretrained(model_name).to(self.device)
            self.tokenizers[f"{self.source_lang}-{tgt_lang}"] = MarianTokenizer.from_pretrained(model_name)

        # Load models for translation from target languages back to source
        for tgt_lang in self.target_langs:
            model_name = f'Helsinki-NLP/opus-mt-{tgt_lang}-{self.source_lang}'
            print(f"Loading {model_name}...")

            self.models[f"{tgt_lang}-{self.source_lang}"] = MarianMTModel.from_pretrained(model_name).to(self.device)
            self.tokenizers[f"{tgt_lang}-{self.source_lang}"] = MarianTokenizer.from_pretrained(model_name)

    def translate(self, texts, src_lang, tgt_lang, batch_size=8):
        """
        Translate a list of texts from source language to target language.

        Args:
            texts: List of texts to translate
            src_lang: Source language code
            tgt_lang: Target language code
            batch_size: Batch size for translation

        Returns:
            List of translated texts
        """
        model = self.models[f"{src_lang}-{tgt_lang}"]
        tokenizer = self.tokenizers[f"{src_lang}-{tgt_lang}"]

        # Process in batches to avoid OOM errors
        translated_texts = []

        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]

            # Tokenize
            encoded = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
            input_ids = encoded["input_ids"].to(self.device)
            attention_mask = encoded["attention_mask"].to(self.device)

            # Translate
            with torch.no_grad():
                translated = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_length=512,
                    num_beams=4,
                    length_penalty=0.6,
                    early_stopping=True
                )

            # Decode
            batch_translated = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
            translated_texts.extend(batch_translated)

        return translated_texts

    def back_translate(self, texts, tgt_lang=None, batch_size=8):
        """
        Perform back-translation: translate from source to target language and back.

        Args:
            texts: List of texts to back-translate
            tgt_lang: Target language code (if None, randomly select from available languages)
            batch_size: Batch size for translation

        Returns:
            List of back-translated texts
        """
        if tgt_lang is None:
            tgt_lang = random.choice(self.target_langs)

        # Translate to target language
        intermediate = self.translate(texts, self.source_lang, tgt_lang, batch_size)

        # Translate back to source language
        back_translated = self.translate(intermediate, tgt_lang, self.source_lang, batch_size)

        return back_translated

# Function to install SentencePiece and other dependencies in Colab
def install_dependencies():
    """Install dependencies required for back-translation."""
    print("Installing required dependencies...")
    import os
    os.system("apt-get update -qq")
    os.system("apt-get install -qq -y cmake build-essential pkg-config libgoogle-perftools-dev")
    os.system("pip install --no-cache-dir sentencepiece")
    os.system("pip install --no-cache-dir transformers torch pandas tqdm")

    # Restart runtime to apply changes
    print("\nRestarting runtime to apply changes...")
    time.sleep(3)

    # This will terminate the current execution - you'll need to run the main function after this
    import os
    os.kill(os.getpid(), 9)

def mount_google_drive():
    """Mount Google Drive to access and save files."""
    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
        return True
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        return False

def augment_imdb_with_back_translation(input_file, output_dir, target_langs=["fr", "de"],
                                      sample_fraction=0.2, batch_size=8):
    """
    Augment the IMDB dataset with back-translation and save to Google Drive.

    Args:
        input_file: Path to the original IMDB CSV file
        output_dir: Directory in Google Drive to save the output files
        target_langs: List of target languages for back-translation
        sample_fraction: Fraction of dataset to augment (to control size)
        batch_size: Batch size for translation
    """
    # Import os here to ensure it's in the local scope
    import os

    # Mount Google Drive if not already mounted
    if not os.path.exists('/content/drive'):
        if not mount_google_drive():
            print("Failed to mount Google Drive. Please check your connection and try again.")
            return

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
        print(f"Created output directory: {output_dir}")

    # Install required dependencies for SentencePiece
    try:
        # Try importing sentencepiece to check if it's installed
        import sentencepiece
        print("SentencePiece already installed.")
    except ImportError:
        print("Installing required dependencies...")
        import os
        os.system("pip install --no-cache-dir sentencepiece")

        # Restart runtime if in Colab (required after installing sentencepiece)
        try:
            import google.colab
            print("\nRestarting runtime to apply changes...")
            time.sleep(3)
            import os
            os.kill(os.getpid(), 9)
            return  # This won't be reached but is here for clarity
        except ImportError:
            print("Not running in Colab, continuing without restart...")

    # Set the output file path
    output_file = os.path.join(output_dir, f"imdb_train_backtranslated_{int(time.time())}.csv")

    # Check if the input file exists
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file not found: {input_file}")

    # Load the IMDB dataset
    print(f"Loading dataset from {input_file}...")
    df = pd.read_csv(input_file)

    # Check for required columns
    if 'review' not in df.columns or 'label' not in df.columns:
        raise ValueError("The dataset must contain 'review' and 'label' columns!")

    # Display dataset statistics
    print(f"Original dataset size: {len(df)} reviews")
    print(f"Class distribution: {df['label'].value_counts().to_dict()}")

    # Initialize the back-translator
    back_translator = BackTranslator(target_langs=target_langs)

    # Sample a subset of the data to augment
    n_samples = int(len(df) * sample_fraction)
    print(f"Sampling {n_samples} reviews for augmentation...")

    # Ensure we have equal representation of both classes
    df_pos = df[df['label'] == 1].sample(n=n_samples//2, random_state=42)
    df_neg = df[df['label'] == 0].sample(n=n_samples//2, random_state=42)
    df_to_augment = pd.concat([df_pos, df_neg], ignore_index=True)

    # Initialize an empty list to store augmented data
    augmented_data = []

    # Save intermediate results periodically
    save_interval = max(len(target_langs) // 2, 1)  # Save after processing each language or two

    # Process each target language
    for lang_idx, lang in enumerate(target_langs):
        print(f"\nGenerating augmentations using {lang} language...")
        reviews = df_to_augment['review'].tolist()
        labels = df_to_augment['label'].tolist()

        # Truncate very long reviews to avoid OOM errors
        max_review_length = 1000  # Characters
        truncated_reviews = [r[:max_review_length] if len(r) > max_review_length else r for r in reviews]

        # Back-translate in batches with a progress bar
        print("Translating to target language...")
        back_translated = []

        # Split into smaller sub-batches for progress tracking
        num_sub_batches = len(truncated_reviews) // batch_size + (1 if len(truncated_reviews) % batch_size > 0 else 0)

        # Add a small delay to ensure tqdm can initialize properly
        time.sleep(0.5)

        for i in tqdm(range(0, len(truncated_reviews), batch_size), total=num_sub_batches):
            batch_reviews = truncated_reviews[i:i+batch_size]
            try:
                batch_back_translated = back_translator.back_translate(
                    batch_reviews, tgt_lang=lang, batch_size=batch_size
                )
                back_translated.extend(batch_back_translated)

                # Print periodic updates
                if (i // batch_size) % 10 == 0 and i > 0:
                    print(f"Processed {i}/{len(truncated_reviews)} reviews")

            except Exception as e:
                print(f"Error during back-translation: {e}")
                # In case of error, use original text
                back_translated.extend(batch_reviews[:(len(batch_reviews))])

        # Create augmented data entries
        for i, (original, translated, label) in enumerate(zip(reviews, back_translated, labels)):
            if original != translated:  # Only add if translation differs from original
                augmented_data.append({
                    'review': translated,
                    'label': label,
                    'technique': f'back_translation_{lang}'
                })

        print(f"Generated {len(augmented_data)} augmented samples so far...")

        # Save intermediate results periodically
        if (lang_idx + 1) % save_interval == 0 or lang_idx == len(target_langs) - 1:
            print(f"Saving intermediate results...")

            # Add technique column to original data for intermediate save
            df_original = df.copy()
            df_original['technique'] = 'original'

            # Create a DataFrame with current augmented data
            if augmented_data:
                df_augmented = pd.DataFrame(augmented_data)

                # Combine original and current augmented data
                df_combined = pd.concat([df_original, df_augmented], ignore_index=True)

                # Save intermediate results to Google Drive
                intermediate_file = os.path.join(
                    output_dir,
                    f"imdb_backtranslated_intermediate_{lang_idx+1}_{int(time.time())}.csv"
                )
                df_combined.to_csv(intermediate_file, index=False)
                print(f"Saved {len(df_augmented)} augmented samples to {intermediate_file}")

    # Add technique column to original data
    df_original = df.copy()
    df_original['technique'] = 'original'

    # Create a DataFrame with augmented data
    if augmented_data:
        df_augmented = pd.DataFrame(augmented_data)

        # Combine original and augmented data
        df_combined = pd.concat([df_original, df_augmented], ignore_index=True)

        # Display augmented dataset statistics
        print(f"\nAugmented dataset size: {len(df_combined)} reviews")
        print(f"Percentage increase: {(len(df_combined) - len(df)) / len(df) * 100:.2f}%")

        # Display techniques distribution
        techniques_count = df_combined['technique'].value_counts()
        print("\nAugmentation techniques distribution:")
        print(techniques_count)

        # Save the augmented dataset to Google Drive
        print(f"Saving augmented dataset to {output_file}...")
        df_combined.to_csv(output_file, index=False)
        print(f"Done! File saved to Google Drive at: {output_file}")

        # Display a few examples of original and back-translated reviews
        print("\nExamples of original and back-translated reviews:")
        for lang in target_langs:
            bt_examples = df_combined[df_combined['technique'] == f'back_translation_{lang}'].head(2)
            for _, row in bt_examples.iterrows():
                print(f"\nBack-translated ({lang}):")
                print(row['review'][:200] + "..." if len(row['review']) > 200 else row['review'])

                # Find the original review
                original_idx = df_to_augment.loc[
                    (df_to_augment['label'] == row['label']) &
                    (df_to_augment.index % len(df_to_augment) ==
                     row.name % len(df_to_augment))
                ].index

                if not original_idx.empty:
                    original_review = df.loc[original_idx[0], 'review']
                    print("\nOriginal:")
                    print(original_review[:200] + "..." if len(original_review) > 200 else original_review)
    else:
        print("No augmented data was generated. Saving original dataset only.")
        df_original.to_csv(output_file, index=False)
        print(f"Done! File saved to Google Drive at: {output_file}")

if __name__ == "__main__":
    # Check if we need to install dependencies first
    try:
        # Try importing sentencepiece to check if it's installed
        import sentencepiece
        print("SentencePiece already installed, skipping installation step.")
    except ImportError:
        print("SentencePiece not found. We'll install it first.")
        install_dependencies()
        # The code will terminate here if dependencies need installation
        sys.exit(0)  # Exit gracefully if dependencies need to be installed

    # Mount Google Drive
    if not os.path.exists('/content/drive'):
        if not mount_google_drive():
            print("Failed to mount Google Drive. Exiting.")
            sys.exit(1)

    # Set the paths for input and output
    input_file = "/content/drive/MyDrive/backtranslation/imdb_train_dataset.csv"
    output_dir = "/content/drive/MyDrive/backtranslation/output"

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
        print(f"Created output directory: {output_dir}")

    # Set the target languages for back-translation
    target_langs = ["fr", "de"]  # Using French and German

    # Set the fraction of the dataset to augment
    sample_fraction = 0.85  # Augment 85% of the dataset

    # Set the batch size for translation
    batch_size = 4

    # Create the input file if it doesn't exist (for testing purposes)
    if not os.path.exists(input_file):
        print(f"Warning: Input file {input_file} not found. Creating a sample file for testing.")

        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(input_file), exist_ok=True)

        # Create a small sample dataset
        sample_data = {
            'review': [
                "This movie was excellent, I loved every minute of it.",
                "Worst film I've ever seen, complete waste of time.",
                "Acting was superb, plot was interesting, highly recommend.",
                "Boring and predictable, don't bother watching.",
            ],
            'label': [1, 0, 1, 0]  # 1 for positive, 0 for negative
        }
        pd.DataFrame(sample_data).to_csv(input_file, index=False)
        print(f"Created sample file {input_file} with 4 reviews.")

    # Run the augmentation
    augment_imdb_with_back_translation(
        input_file=input_file,
        output_dir=output_dir,
        target_langs=target_langs,
        sample_fraction=sample_fraction,
        batch_size=batch_size
    )

SentencePiece already installed, skipping installation step.
SentencePiece already installed.
Loading dataset from /content/drive/MyDrive/backtranslation/imdb_train_dataset.csv...
Original dataset size: 25000 reviews
Class distribution: {1: 12500, 0: 12500}
Using CUDA device: NVIDIA A100-SXM4-40GB
Loading translation models...
Loading Helsinki-NLP/opus-mt-en-fr...


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Loading Helsinki-NLP/opus-mt-en-de...
Loading Helsinki-NLP/opus-mt-fr-en...
Loading Helsinki-NLP/opus-mt-de-en...
Sampling 21250 reviews for augmentation...

Generating augmentations using fr language...
Translating to target language...


  0%|          | 11/5313 [00:51<6:53:34,  4.68s/it]

Processed 40/21250 reviews


...

Processed 21200/21250 reviews


100%|█████████▉| 5311/5313 [7:22:46<00:09,  4.84s/it]

Processed 21240/21250 reviews


100%|██████████| 5313/5313 [7:22:54<00:00,  5.00s/it]


Generated 42499 augmented samples so far...
Saving intermediate results...
Saved 42499 augmented samples to /content/drive/MyDrive/backtranslation/output/imdb_backtranslated_intermediate_2_1742435484.csv

Augmented dataset size: 67499 reviews
Percentage increase: 170.00%

Augmentation techniques distribution:
technique
original               25000
back_translation_fr    21250
back_translation_de    21249
Name: count, dtype: int64
Saving augmented dataset to /content/drive/MyDrive/backtranslation/output/imdb_train_backtranslated_1742384204.csv...
Done! File saved to Google Drive at: /content/drive/MyDrive/backtranslation/output/imdb_train_backtranslated_1742384204.csv

Examples of original and back-translated reviews:

Back-translated (fr):
This film is an eye-opener for those who can only the glamorous lifestyles of the stars. It tells you how people who would like to do good are not able to do it. Moreover, the scene of the bomb explos...

Original:
"Der Todesking" is not exactly the 